Mô hình phân loại bệnh tim 5 class (mức độ từ nặng đến nhẹ) dựa trên 14 features đầu vào sử dụng Softmax Regression

Dataset lấy từ Kaggle, gồm 920 samples. 

Thực hiện train trên 2500 epochs với learning rate 0.0001
Cho hiệu quả acccuracy ~56% trên tập test.

-> Hiệu quả tương đối hạn chế vì Softmax Regression vẫn dựa vào nền tảng tuyến tính, phân loại dữ liệu 5 class đầu ra còn hạn chế

Đỗ Văn Hoàng Anh


In [282]:
from math import exp
from math import log
import numpy as np  
import pandas as pd
import kagglehub



path = kagglehub.dataset_download("redwankarimsony/heart-disease-data")
print("Path to dataset files:", path)
path=path+'/heart_disease_uci.csv'
data=pd.read_csv(path)

data=data.drop('id', axis=1)


#data mapping
data['fbs'] = data['fbs'].map({True: 1, False: 0})
data['sex']=data['sex'].map({'Male':0,'Female':1})
data['exang']=data['exang'].map({True:1,False:0})
data['dataset']=data['dataset'].map({'Cleveland':1,'Hungary':0,'VA Long Beach':2,'Switzerland':3}) 
data['cp']=data['cp'].map({'asymptomatic':1,'typical angina':0,'atypical angina':2,'non-anginal':3})
data['restecg']=data['restecg'].map({'lv hypertrophy':0,'normal':1,'st-t abnormality':2})
data['slope']=data['slope'].map({'downsloping':0,'flat':1,'upsloping':2})
data['thal']=data['thal'].map({'fixed defect':0,'normal':1,'reversable defect':2})


# fill dữ liệu bị missing
for col in ['trestbps', 'chol', 'thalch', 'oldpeak']:
    data[col] = data[col].fillna(data[col].mean())

for col in ['fbs', 'restecg', 'exang', 'slope', 'ca', 'thal']:
    data[col] = data[col].fillna(data[col].mode()[0])


Path to dataset files: C:\Users\dhoan\.cache\kagglehub\datasets\redwankarimsony\heart-disease-data\versions\6


In [ ]:
def Z_calc(x,w,b):
    z=np.dot(w,x.T)+b.T
    return z
    #z(5,1)
def softmax(z):
    sum_exp = np.sum(np.exp(z), axis=0)
    prob = np.exp(z) / sum_exp
    return prob.T
    #prob()
def argmax(prob):
    return np.argmax(prob, axis=0)
def test(w,b,x,y):
    true_pred=0
    for i in range(len(x)):
        z = Z_calc(x, w, b)
        prob = softmax(z)
        res=argmax(prob)
        if res==y[i]:
            true_pred+=1
    accuracy=true_pred/len(x)
    return accuracy
    
def one_hot_encode(y):
    res=np.zeros((1,5))
    res[0,y]=1
    return res


def loss_function(y_true, y_pred):
    epsilon = 1e-15
    y_pred = np.clip(y_pred, epsilon, 1 - epsilon)
    return -np.sum(y_true * np.log(y_pred))


def update_weights(x, y_true, y_pred, w, b, learning_rate):
    dz = y_pred - y_true 
    dw = np.dot(x.T, dz) 
    db = dz
    w -= (learning_rate * dw).T
    b -= learning_rate * db
    return w, b


def train(x_train, y_train, w, b, epochs, learning_rate):
    for epoch in range(epochs):
        loss=0
        print("Epoch:", epoch+1)
        for i in range(len(x_train)):
            z = Z_calc(x_train[i, 0::].reshape(1,14), w, b)
            prob = softmax(z)
            y_true = one_hot_encode(y_train[i,0])
            
            
            w, b = update_weights(x_train[i, 0::].reshape(1,14), y_true, prob, w, b, learning_rate)
            loss+=loss_function(y_true, prob)
        print("Loss:", loss/len(x_train))
    
       
    


In [ ]:
from sklearn.model_selection import train_test_split


w = 0.01 * np.random.randn(5, 14)
b=np.zeros((1,5))


X = data.drop('num', axis=1).values
Y = data['num'].values

#chuẩn hóa dữ liệu
mean = np.mean(X, axis=0)
std = np.std(X, axis=0)
X = (X - mean) / std
#tách train test

x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)


epochs=2500
learning_rate=0.0001
y_train=y_train.reshape(736,1)
train(x_train, y_train, w, b, epochs, learning_rate)

Epoch: 1
Loss: 1.5964640674423516
Epoch: 2
Loss: 1.5625432421581265
Epoch: 3
Loss: 1.5307680687794651
Epoch: 4
Loss: 1.5010477418806618
Epoch: 5
Loss: 1.4732867033703336
Epoch: 6
Loss: 1.4473853369116085
Epoch: 7
Loss: 1.4232410408007532
Epoch: 8
Loss: 1.4007495392370772
Epoch: 9
Loss: 1.379806289663702
Epoch: 10
Loss: 1.360307860677232
Epoch: 11
Loss: 1.3421531836553462
Epoch: 12
Loss: 1.325244614343299
Epoch: 13
Loss: 1.3094887721954922
Epoch: 14
Loss: 1.2947971514609367
Epoch: 15
Loss: 1.281086517148925
Epoch: 16
Loss: 1.2682791110956346
Epoch: 17
Loss: 1.256302699383901
Epoch: 18
Loss: 1.2450904938064962
Epoch: 19
Loss: 1.2345809784006954
Epoch: 20
Loss: 1.2247176686064818
Epoch: 21
Loss: 1.215448826298686
Epoch: 22
Loss: 1.2067271494961174
Epoch: 23
Loss: 1.1985094513800407
Epoch: 24
Loss: 1.1907563395841547
Epoch: 25
Loss: 1.1834319036351615
Epoch: 26
Loss: 1.1765034159272283
Epoch: 27
Loss: 1.16994104965577
Epoch: 28
Loss: 1.1637176156437616
Epoch: 29
Loss: 1.1578083188886392
Ep

In [ ]:
def test_accuracy(x, y, w, b):
    z = Z_calc(x, w, b)
    prob = softmax(z)
    predictions = np.argmax(prob, axis=1)
    y_true = y.flatten()
    matches = (predictions == y_true)
    accuracy = np.mean(matches)
    return accuracy

acc_train = test_accuracy(x_train, y_train, w, b)
print(f"Độ chính xác trên tập TRAIN: {acc_train * 100:.2f}%")

acc_test = test_accuracy(x_test, y_test, w, b)
print(f"Độ chính xác trên tập TEST:  {acc_test * 100:.2f}%")

Độ chính xác trên tập TRAIN: 60.73%
Độ chính xác trên tập TEST:  55.98%


In [298]:
!git add SMR.ipynb
!git commit -m "update"
!git push origin main

[main 17df676] update
 1 file changed, 10 insertions(+), 56 deletions(-)


To https://github.com/hoanganhdo2316-png/AIO.git
   2c5371e..17df676  main -> main
